In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import re
import os
import requests
import dateutil
from datetime import datetime

pd.set_option("display.max_rows", 1000)

plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

In [2]:
overview_page_urls = ["http://www.kicker.de/news/fussball/weltmeisterschaft/spiele/weltmeisterschaft/2018/7/0/spieltag.html","http://www.kicker.de/news/fussball/weltmeisterschaft/spiele/weltmeisterschaft/2018/8/0/spieltag.html"]

In [3]:
url_list = []
tactical_url_list = []
names_list = []

for url in overview_page_urls:
    #open each of the overview pages
    requested_page = Request(url, headers={'User-Agent': 'Chrome/64.0'})
    webpage = urlopen(requested_page).read()
    overview_page = BeautifulSoup(webpage, "html.parser")
    
    #Find the links to all the pages that contain an analysis
    for a in overview_page.findAll('a', {'class': 'link'}):
        if a.text == "Analyse":
            print(a['href'])
            url_list.append(a['href'])
        if  a.text == "Spielbericht":
            print(a['href'])
            url_list.append(a['href'])
    
    #For each of the found links, replace part of the URL to head to the tactical setup and save it as a new URL
    for url in url_list:
        modified_url = url.replace('spielanalyse','taktische-aufstellung').replace('spielbericht','taktische-aufstellung')
        full_url = 'http://www.kicker.de'+modified_url
        print(full_url)
        tactical_url_list.append(full_url)
        
    for tactical_url in tactical_url_list:
        #open the URL for each game's tactical set up listing all players and exchanged players
        requested_page_two = Request(tactical_url)
        webpage_two = urlopen(requested_page_two).read()
        overview_page_two = BeautifulSoup(webpage_two, "html.parser")

        #find all the names listed on the page and add them to the list
        #search by div-id as that is the only way to exclude the players on the Ersatzbank
        for div in overview_page_two.findAll('div', {'id':'ctl00_PlaceHolderHalf_aufstellunghalf_ausstellungHeim'}):
            for a in div.findAll('a', {'class':'link_noicon'}):
                print(a.text)
                names_list.append(a.text)

        for div in overview_page_two.findAll('div', {'id':'ctl00_PlaceHolderHalf_aufstellunghalf_einwechslungenHeim'}):
            for a in div.findAll('a', {'class':'link_noicon'}):
                print(a.text)
                names_list.append(a.text)

        for div in overview_page_two.findAll('div', {'id':'ctl00_PlaceHolderHalf_aufstellunghalf_ausstellungAusw'}):
            for a in div.findAll('a', {'class':'link_noicon'}):
                print(a.text)
                names_list.append(a.text)

        for div in overview_page_two.findAll('div', {'id':'ctl00_PlaceHolderHalf_aufstellunghalf_einwechslungenAusw'}):
            for a in div.findAll('a', {'class':'link_noicon'}):
                print(a.text)
                names_list.append(a.text)

/news/fussball/weltmeisterschaft/spiele/weltmeisterschaft/2018/7/3070468/spielanalyse_belgien_england.html
http://www.kicker.de/news/fussball/weltmeisterschaft/spiele/weltmeisterschaft/2018/7/3070468/taktische-aufstellung_belgien_england.html
Courtois
Alderweireld
Kompany
Vertonghen
Meunier
Tielemans
Witsel
Chadli
De Bruyne
E. Hazard
R. Lukaku
Vermaelen
Chadli
Mertens
R. Lukaku
Dembelé
Tielemans
Pickford
P. Jones
Stones
Maguire
Trippier
Dier
D. Rose
Loftus-Cheek
Delph
Sterling
H. Kane
Lingard
D. Rose
Rashford
Sterling
Alli
Loftus-Cheek
/news/fussball/weltmeisterschaft/spiele/weltmeisterschaft/2018/8/3070469/spielbericht_frankreich_kroatien.html
http://www.kicker.de/news/fussball/weltmeisterschaft/spiele/weltmeisterschaft/2018/7/3070468/taktische-aufstellung_belgien_england.html
http://www.kicker.de/news/fussball/weltmeisterschaft/spiele/weltmeisterschaft/2018/8/3070469/taktische-aufstellung_frankreich_kroatien.html
Courtois
Alderweireld
Kompany
Vertonghen
Meunier
Tielemans
Witsel
Chadl

In [4]:
df = pd.DataFrame(names_list)
df.head()

,0
0,Courtois
1,Alderweireld
2,Kompany
3,Vertonghen
4,Meunier


In [5]:
df.to_csv('data/output/players_played_2018_finals_raw.csv')

In [6]:
print(len(df))
df_unique = df.drop_duplicates()
print(len(df_unique))

134
57


In [7]:
df_unique.to_csv('data/output/players_played_2018_finals_unique.csv')